In [10]:
from lsst.sims.ocs.observatory import MainObservatory
from lsst.sims.ocs.configuration import ObservingSite, Observatory

In [11]:
site = ObservingSite()

In [12]:
mo = MainObservatory(site)
mo.configure(Observatory())

In [13]:
mo.start_night(10, 3652)

In [22]:
dir(mo.model.current_state)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'alt',
 'alt_rad',
 'ang',
 'ang_rad',
 'az',
 'az_rad',
 'dec',
 'dec_rad',
 'domalt',
 'domalt_peakspeed',
 'domalt_peakspeed_rad',
 'domalt_rad',
 'domaz',
 'domaz_peakspeed',
 'domaz_peakspeed_rad',
 'domaz_rad',
 'filter',
 'mountedfilters',
 'pa',
 'pa_rad',
 'ra',
 'ra_rad',
 'rot',
 'rot_rad',
 'set',
 'set_position',
 'swap_filter',
 'telalt',
 'telalt_peakspeed',
 'telalt_peakspeed_rad',
 'telalt_rad',
 'telaz',
 'telaz_peakspeed',
 'telaz_peakspeed_rad',
 'telaz_rad',
 'telrot',
 'telrot_peakspeed',
 'telrot_peakspeed_rad',
 'telrot_rad',
 'time',
 'tracking',
 'unmountedfilters']

In [31]:
mo.model.current_state.tracking

False

In [ ]:
# Need to make dummy classes for:
# SALPY_scheduler.targetC
# SALPY_scheduler.observationC
# and a TimeHandler object

# for the topic:
#def from_topic(cls, topic):
#        """Alternate initializer.
#
#        Parameters
#        ----------
#        topic : SALPY_scheduler.targetC
#            The target topic instance.
#
#        Returns
#        -------
#        :class:`.Target`
#        """
#        return cls(topic.targetId, topic.fieldId, topic.filter, math.radians(topic.ra),
#                   math.radians(topic.dec), math.radians(topic.angle), topic.num_exposures,
#                   topic.exposure_times)

